In [1]:
import warnings
import os
from pathlib import Path 
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import anndata
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import decoupler as dc
import itertools
from scipy import sparse
#import lib.data_helpers as dh

#import lib.scanpy_helpers as sh
#import lib.pl.util as pu

/home/kvalem/.local/lib/python3.11/site-packages/numba/core/cpu.py:99: UserWarning: Numba extension module 'awkward.numba' failed to load due to 'ModuleNotFoundError(No module named 'importlib_metadata')'.
  numba.core.entrypoints.init_all()


In [2]:
path = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/pseudobulk/"
resDir_figures = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/figures/LUAD_DE/"
resDir_tables = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/"
input_path = f"{path}/paired_adata_clean.h5ad"

In [3]:
#adata = sc.read_h5ad(input_path) 

In [4]:
adata = sc.read_h5ad("/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/old/pseudobulk/paired_adata_clean_luad_all_cell_type.h5ad")

In [5]:
# Define the cell types to keep
selected_cell_types = {
    'macrophage',
    'CD4+ T cell',
    'CD8+ T cell',
    'B cell',
    'malignant cell',
    'regulatory T cell',
    'epithelial cell',
    'neutrophil',
    'dendritic cell',
    'plasma cell',
    'endothelial cell',
    'fibroblast'
}

# Mapping for synonymous cell types in adata
cell_type_mapping = {
    'fibroblast of lung': 'fibroblast',
    'vein endothelial cell': 'endothelial cell',
    'capillary endothelial cell': 'endothelial cell',
    'pulmonary artery endothelial cell': 'endothelial cell',
    'endothelial cell of lymphatic vessel': 'endothelial cell',
    'conventional dendritic cell': 'dendritic cell',
    'CD1c-positive myeloid dendritic cell': 'dendritic cell',
    'plasmacytoid dendritic cell': 'dendritic cell',
    'bronchus fibroblast of lung': 'fibroblast'
}

# Standardize cell types based on mapping
adata.obs['standardized_cell_type'] = adata.obs['cell_type'].replace(cell_type_mapping)

# Filter adata to keep only selected cell types
adata = adata[adata.obs['standardized_cell_type'].isin(selected_cell_types)].copy()

# Verify result
print(adata.obs['standardized_cell_type'].value_counts())


standardized_cell_type
macrophage           104271
CD4+ T cell           94503
CD8+ T cell           72603
epithelial cell       21039
malignant cell        16939
B cell                16853
regulatory T cell     16785
dendritic cell        16702
endothelial cell      16150
plasma cell           15096
neutrophil             6146
fibroblast             4674
Name: count, dtype: int64


## There are malignant cells in normal_adjacent samples
Comming from 59 different patients and 14 datasets
I cannot give an explanation for this so i remove this "mislalbeled" cells 

In [6]:
grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')

In [7]:
grouped

,standardized_cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,4957
1,epithelial cell,tumor_primary,16082
2,macrophage,normal_adjacent,65453
3,macrophage,tumor_primary,38818
4,B cell,normal_adjacent,2888
5,B cell,tumor_primary,13965
6,dendritic cell,normal_adjacent,6752
7,dendritic cell,tumor_primary,9950
8,CD4+ T cell,normal_adjacent,45654
9,CD4+ T cell,tumor_primary,48849


In [8]:
# Create a Boolean mask for the cells that you want to keep
mask = ~((adata.obs['origin'] == 'normal_adjacent') & (adata.obs['standardized_cell_type'] == 'malignant cell'))

# Subset the adata object using the mask to exclude the specific cells
adata = adata[mask].copy()

# Now `adata_filtered` contains all cells except those with origin == 'normal_adjacent' and cell_type == 'malignant cell'


In [9]:
grouped = adata.obs.groupby(['standardized_cell_type', 'origin']).size().reset_index(name='cell_count')
grouped

,standardized_cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,4957
1,epithelial cell,tumor_primary,16082
2,macrophage,normal_adjacent,65453
3,macrophage,tumor_primary,38818
4,B cell,normal_adjacent,2888
5,B cell,tumor_primary,13965
6,dendritic cell,normal_adjacent,6752
7,dendritic cell,tumor_primary,9950
8,CD4+ T cell,normal_adjacent,45654
9,CD4+ T cell,tumor_primary,48849


In [10]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [11]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD1c-positive myeloid dendritic cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'bronchus fibroblast of lung',
 'capillary endothelial cell',
 'conventional dendritic cell',
 'dendritic cell',
 'endothelial cell of lymphatic vessel',
 'epithelial cell',
 'fibroblast of lung',
 'macrophage',
 'neutrophil',
 'plasma cell',
 'plasmacytoid dendritic cell',
 'pulmonary artery endothelial cell',
 'regulatory T cell',
 'vein endothelial cell'}

In [12]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [13]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD1c-positive myeloid dendritic cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'bronchus fibroblast of lung',
 'capillary endothelial cell',
 'conventional dendritic cell',
 'dendritic cell',
 'endothelial cell of lymphatic vessel',
 'epithelial cell',
 'fibroblast of lung',
 'macrophage',
 'neutrophil',
 'plasma cell',
 'plasmacytoid dendritic cell',
 'pulmonary artery endothelial cell',
 'regulatory T cell',
 'vein endothelial cell'}

In [14]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [15]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [16]:
adata

AnnData object with n_obs × n_vars = 401187 × 17811
    obs: 'sample', 'uicc_stage', 'ever_smoker', 'age', 'donor_id', 'origin', 'dataset', 'ann_fine', 'cell_type_predicted', 'doublet_status', 'leiden', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'ann_coarse', 'cell_type_tumor', 'tumor_stage', 'EGFR_mutation', 'TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation', 'KRAS_mutation', 'ROS_mutation', 'origin_fine', 'study', 'platform', 'cell_type_major', 'cell_type_neutro', 'cell_type_neutro_coarse', 'suspension_type', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'standardized_cell_type'
    var: 'is_highly_variable', 'm

In [17]:
adata_male = adata[adata.obs["sex"]=="male"]

In [18]:
adata_female = adata[adata.obs["sex"]=="female"]

## Prepare for pseudobulk

In [19]:
contrasts = [
        dict(var = "origin", condition = "tumor_primary", reference = "normal_adjacent"),
    ]

In [20]:
contrasts

[{'var': 'origin',
  'condition': 'tumor_primary',
  'reference': 'normal_adjacent'}]

In [21]:
contrasts[0]["condition"].replace(" ", "_") + "_vs_" + contrasts[0]["reference"].replace(" ", "_")

'tumor_primary_vs_normal_adjacent'

In [22]:
resDir_tables

'/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/'

In [23]:
for contrast in contrasts:
    name = contrast["condition"].replace(" ", "_") + "_vs_" + contrast["reference"].replace(" ", "_")
    contrast["name"] = name
    res_dir = Path(resDir_tables, name, "tables")
    os.makedirs(resDir_tables, mode = 0o750, exist_ok = True)
    contrast["res_dir"] = resDir_tables

In [24]:
cell_type_class = "standardized_cell_type"

In [25]:
contrast["res_dir"]

'/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/'

In [26]:
pdata = dc.get_pseudobulk(adata,
                          sample_col='sample',
                          groups_col=cell_type_class,
                          layer='count',
                          mode='sum',
                          min_cells=10,
                          min_counts=1000
                         )

In [27]:
pdata.X = pdata.X.astype(int)

In [28]:
pdata.var_names = pdata.var["feature_name"].astype(str)

In [29]:
pdata_obs_df = pdata.obs

In [30]:
pdata

AnnData object with n_obs × n_vars = 1269 × 17811
    obs: 'sample', 'standardized_cell_type', 'uicc_stage', 'ever_smoker', 'donor_id', 'origin', 'dataset', 'tumor_stage', 'study', 'platform', 'suspension_type', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'psbulk_n_cells', 'psbulk_counts'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    layers: 'psbulk_props'

In [31]:
# Group by patient and filter for patients who have samples in both 'tumor' and 'normal_adjacent' conditions
patients_with_both_conditions = pdata_obs_df.groupby('donor_id')['origin'].apply(lambda x: set(x) >= {'tumor_primary', 'normal_adjacent'})

In [32]:
# Get the patient IDs that meet the condition
patients_with_both_conditions_ids = patients_with_both_conditions[patients_with_both_conditions].index

In [33]:
pdata_tumor_normal = pdata[pdata.obs['donor_id'].isin(patients_with_both_conditions_ids)]

In [34]:
cell_type_class

'standardized_cell_type'

In [35]:
## Run deseq2 on pseudobulk all cell types
#cell_types = pdata_tumor_normal.obs[cell_type_class].unique()

In [36]:
contrast["reference"]

'normal_adjacent'

In [37]:
contrast["condition"]

'tumor_primary'

In [38]:
contrast["name"]

'tumor_primary_vs_normal_adjacent'

In [39]:
pdata_tumor_normal.obs['sex'] = pdata_tumor_normal.obs['sex'].astype('category')

/tmp/ipykernel_2960518/2235016355.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  pdata_tumor_normal.obs['sex'] = pdata_tumor_normal.obs['sex'].astype('category')


In [40]:
cpus=16

In [41]:
pdata_tumor_normal

AnnData object with n_obs × n_vars = 1268 × 17811
    obs: 'sample', 'standardized_cell_type', 'uicc_stage', 'ever_smoker', 'donor_id', 'origin', 'dataset', 'tumor_stage', 'study', 'platform', 'suspension_type', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'psbulk_n_cells', 'psbulk_counts'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    layers: 'psbulk_props'

In [42]:
pdata_tumor_normal.obs["standardized_cell_type"]= pdata_tumor_normal.obs["standardized_cell_type"].replace(['epithelial cell of lung','multi-ciliated epithelial cell',], 'epithelial cell')
pdata_tumor_normal.obs["standardized_cell_type"]= pdata_tumor_normal.obs["standardized_cell_type"].replace(['alveolar macrophage'], 'macrophage')
pdata_tumor_normal.obs["standardized_cell_type"]= pdata_tumor_normal.obs["standardized_cell_type"].replace(['CD4-positive, alpha-beta T cell'], 'CD4+ T cell')
pdata_tumor_normal.obs["standardized_cell_type"]= pdata_tumor_normal.obs["standardized_cell_type"].replace(['CD8-positive, alpha-beta T cell'], 'CD8+ T cell')

In [43]:
pdata_tumor_normal.obs["origin"].value_counts()

origin
tumor_primary      708
normal_adjacent    560
Name: count, dtype: int64

In [44]:
pdata_only_tumor = pdata_tumor_normal[pdata_tumor_normal.obs["origin"]=="tumor_primary"]

In [45]:
pdata_only_tumor

View of AnnData object with n_obs × n_vars = 708 × 17811
    obs: 'sample', 'standardized_cell_type', 'uicc_stage', 'ever_smoker', 'donor_id', 'origin', 'dataset', 'tumor_stage', 'study', 'platform', 'suspension_type', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'psbulk_n_cells', 'psbulk_counts'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    layers: 'psbulk_props'

In [46]:
resDir_tables

'/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/'

In [47]:
cell_types = pdata_only_tumor.obs[cell_type_class].unique()

In [48]:
pdata_tumor_normal_male = pdata_tumor_normal[pdata_tumor_normal.obs["sex"]=="male"]

In [64]:
pdata_tumor_normal_female = pdata_tumor_normal[pdata_tumor_normal.obs["sex"]=="female"]

In [49]:
pdata_tumor_normal_male.obs["origin"] = pdata_tumor_normal_male.obs["origin"].str.replace("_", "-", regex=False)


/tmp/ipykernel_2960518/680501246.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  pdata_tumor_normal_male.obs["origin"] = pdata_tumor_normal_male.obs["origin"].str.replace("_", "-", regex=False)


In [66]:
pdata_tumor_normal_female.obs["origin"] = pdata_tumor_normal_female.obs["origin"].str.replace("_", "-", regex=False)


/tmp/ipykernel_2960518/307224016.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  pdata_tumor_normal_female.obs["origin"] = pdata_tumor_normal_female.obs["origin"].str.replace("_", "-", regex=False)


In [68]:
pdata_tumor_normal_male = pdata_tumor_normal_male[~pdata_tumor_normal_male.obs["standardized_cell_type"].isin(["malignant cell"])]
pdata_tumor_normal_female= pdata_tumor_normal_female[~pdata_tumor_normal_female.obs["standardized_cell_type"].isin(["malignant cell"])]

In [69]:
contrast["reference"] = "normal-adjacent"
contrast["condition"] = "tumor-primary"
contrast["name"] = "tumor-primary_vs_normal-adjacent"

## Pydeseq tumor vs normal MALE

In [115]:
import pdb
for contrast in contrasts:
    de_res = {}

    for ct in cell_types:
        print(ct)
        
        pb_ct = pdata_tumor_normal_male[pdata_tumor_normal_male.obs[cell_type_class] == ct].copy()

     
        #pdb.set_trace()
        if len(set(pb_ct.obs["origin"]).intersection([contrast["reference"], contrast["condition"]])) < 1:
             print(
                 "Not running DEseq for: "
                  + ct
                  + " : only present in: "
                  + str(set(pb_ct.obs["sex"]).intersection([contrast["reference"], contrast["condition"]]))
             )
             continue
        #pdb.set_trace()
                     
        dds = DeseqDataSet(
           adata=pb_ct,
           design_factors=[contrast["var"]],
           ref_level=[contrast["var"], 'normal-adjacent'],
           refit_cooks=True,
           n_cpus=cpus,
           )
             
        

        # Compute LFCs
        dds.deseq2()

 

        

        # Extract contrast
        stat_res = DeseqStats(
              dds,
              contrast=[contrast["var"],'tumor-primary', 'normal-adjacent'],
              #n_cpus=cpus,
          )
   
    

        # Compute Wald test
        stat_res.summary()

        # Shrink LFCs

        coeff = contrast["var"] + "_" + contrast["name"]
        stat_res.lfc_shrink(coeff=coeff)

        # Register cell type results
        de_res[ct] = stat_res.results_df
        de_res[ct]["standardized_cell_type"] = ct
        de_res[ct]["feature_name"] = stat_res.results_df.index.values

        de_res[ct].drop(columns=["feature_name"]).to_csv(
                Path(
                    contrast["res_dir"],
                    ct.replace(" ", "_") + "_" + contrast["name"] + "_deseq.tsv",
                ),
                sep="\t",
            )

        # Register results for current contrast
        contrast["de_res"] = de_res
        contrast["de_res_all"] = pd.concat([df.assign(cell_type=ct) for ct, df in de_res.items()])

B cell


Fitting size factors...
... done in 0.03 seconds.

Fitting dispersions...
... done in 1.19 seconds.

Fitting dispersion trend curve...
... done in 0.35 seconds.

Fitting MAP dispersions...
... done in 1.43 seconds.

Fitting LFCs...
... done in 1.49 seconds.

Calculating cook's distance...
... done in 0.06 seconds.

Replacing 59 outlier genes.

Fitting dispersions...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.02 seconds.

Running Wald tests...
... done in 8.70 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
feature_name                                                                  
A1BG          4.613264        0.000056  0.323332  0.000174  0.999861  0.999978
A1BG-AS1      0.591671       -0.200691  0.570672 -0.351676  0.725081       NaN
A2M           1.206051       -0.223805  0.620775 -0.360525  0.718455       NaN
A2M-AS1       0.082406       -1.195479  1.465479 -0.815760  0.414638       NaN
A2ML1         0.157295       -1.658056  1.488754 -1.113721  0.265399       NaN
...                ...             ...       ...       ...       ...       ...
ZXDC          0.928493       -0.355715  0.578193 -0.615219  0.538410       NaN
ZYG11A        0.066189       -0.837893  1.683108 -0.497825  0.618607       NaN
ZYG11B        1.394873        0.450700  0.493065  0.914079  0.360675       NaN
ZYX           3.686372        0.780150  0.348659  2.2

/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
... done in 2.01 seconds.

Fitting size factors...


Shrunk log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
feature_name                                                                  
A1BG          4.613264        0.000005  0.052210  0.000174  0.999861  0.999978
A1BG-AS1      0.591671        0.000091  0.063279 -0.351676  0.725081       NaN
A2M           1.206051       -0.000674  0.053568 -0.360525  0.718455       NaN
A2M-AS1       0.082406        0.000124  0.060909 -0.815760  0.414638       NaN
A2ML1         0.157295       -0.000482  0.046209 -1.113721  0.265399       NaN
...                ...             ...       ...       ...       ...       ...
ZXDC          0.928493       -0.000956  0.053715 -0.615219  0.538410       NaN
ZYG11A        0.066189        0.000512  0.077715 -0.497825  0.618607       NaN
ZYG11B        1.394873       -0.005089  0.048999  0.914079  0.360675       NaN
ZYX           3.686372        0.006723  0.0617

...

Fitting size factors...
... done in 0.03 seconds.

Fitting dispersions...
... done in 1.53 seconds.

Fitting dispersion trend curve...
... done in 0.36 seconds.

Fitting MAP dispersions...
... done in 1.43 seconds.

Fitting LFCs...
... done in 1.54 seconds.

Calculating cook's distance...
... done in 0.06 seconds.

Replacing 1377 outlier genes.

Fitting dispersions...
... done in 0.17 seconds.

Fitting MAP dispersions...
... done in 0.15 seconds.

Fitting LFCs...
... done in 0.14 seconds.

Running Wald tests...
... done in 8.75 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
feature_name                                                            
A1BG           7.033111       -0.142249  0.394206 -0.360849  0.718212   
A1BG-AS1       1.485339        0.159139  0.623456  0.255254  0.798527   
A2M            1.273227        0.290248  0.728302  0.398527  0.690242   
A2M-AS1        0.173113       -1.321383  1.405063 -0.940444  0.346990   
A2ML1          0.176934       -2.254269  2.922899 -0.771244  0.440562   
...                 ...             ...       ...       ...       ...   
ZXDC           1.019239        0.850460  0.792190  1.073555  0.283022   
ZYG11A         0.014163       -2.067385  3.053920 -0.676961  0.498431   
ZYG11B         3.356246       -0.176350  0.442509 -0.398524  0.690244   
ZYX           14.740712       -0.403523  0.279140 -1.445596  0.148290   
ZZEF1          3.895481        0.452235  0.394

/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))


Shrunk log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
feature_name                                                            
A1BG           7.033111   -9.155528e-07  0.001640 -0.360849  0.718212   
A1BG-AS1       1.485339    6.611014e-07  0.001920  0.255254  0.798527   
A2M            1.273227    8.387755e-07  0.001885  0.398527  0.690242   
A2M-AS1        0.173113    1.044228e-07  0.002154 -0.940444  0.346990   
A2ML1          0.176934   -7.613284e-08  0.001425 -0.771244  0.440562   
...                 ...             ...       ...       ...       ...   
ZXDC           1.019239    2.913140e-06  0.002257  1.073555  0.283022   
ZYG11A         0.014163    1.193448e-07  0.002688 -0.676961  0.498431   
ZYG11B         3.356246   -9.622742e-07  0.001682 -0.398524  0.690244   
ZYX           14.740712   -4.786789e-06  0.001580 -1.445596  0.148290   
ZZEF1          3.895481    3.102382e-06

... done in 3.28 seconds.



In [116]:
contrast["de_res_all"]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,standardized_cell_type,feature_name,cell_type
feature_name,,,,,,,,,
A1BG,4.613264,5.484595e-06,0.052210,0.000174,0.999861,0.999978,B cell,A1BG,B cell
A1BG-AS1,0.591671,9.067974e-05,0.063279,-0.351676,0.725081,NaN,B cell,A1BG-AS1,B cell
A2M,1.206051,-6.738284e-04,0.053568,-0.360525,0.718455,NaN,B cell,A2M,B cell
A2M-AS1,0.082406,1.239167e-04,0.060909,-0.815760,0.414638,NaN,B cell,A2M-AS1,B cell
A2ML1,0.157295,-4.821732e-04,0.046209,-1.113721,0.265399,NaN,B cell,A2ML1,B cell
...,...,...,...,...,...,...,...,...,...
ZXDC,1.019239,2.913140e-06,0.002257,1.073555,0.283022,NaN,regulatory T cell,ZXDC,regulatory T cell
ZYG11A,0.014163,1.193448e-07,0.002688,-0.676961,0.498431,NaN,regulatory T cell,ZYG11A,regulatory T cell
ZYG11B,3.356246,-9.622742e-07,0.001682,-0.398524,0.690244,0.958322,regulatory T cell,ZYG11B,regulatory T cell


In [118]:
resDir = resDir_tables

In [119]:
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Loop through each cell type and save its data to CSV
for cell_type in cell_types:
    # Filter the DataFrame for the current cell type
    ct_all_deg = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]
    
    # Remove spaces from cell_type for the filename
    cell_type_filename = cell_type.replace(" ", "")
    
    # Save the full data for the cell type
    filename_deg = f"{cell_type_filename}_deg.csv"
    filepath_deg = os.path.join(resDir, filename_deg)
    ct_all_deg.to_csv(filepath_deg, index=False)
    print(f"Saved full {cell_type} data to {filepath_deg}")
    
    # Apply additional filtering for significant results
    filtered_df = ct_all_deg[(ct_all_deg['padj'] < 0.1) & (ct_all_deg['log2FoldChange'].abs() > 1)]
    
    # Save the filtered significant data for the cell type
    filename_sig_deg = f"{cell_type_filename}_sig_deg.csv"
    filepath_sig_deg = os.path.join(resDir, filename_sig_deg)
    filtered_df.to_csv(filepath_sig_deg, index=False)
    print(f"Saved significant {cell_type} data to {filepath_sig_deg}")

Saved full B cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/Bcell_deg.csv
Saved significant B cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/Bcell_sig_deg.csv
Saved full CD4+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD4+Tcell_deg.csv
Saved significant CD4+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD4+Tcell_sig_deg.csv
Saved full CD8+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD8+Tcell_deg.csv
Saved significant CD8+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD8+Tcell_sig_deg.csv
Saved full dendritic cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/dendriticcell_deg.csv
Saved significant d

In [120]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming `contrast` and `dc` are already defined and set up in your environment
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Create a PDF file to save all volcano plots
with PdfPages(f"{resDir_figures}/volcano_plots_all_cell_types.pdf") as pdf:
    for cell_type in cell_types:
        # Filter results for the current cell type
        results_df = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]

        # Generate the volcano plot
        plt.figure(figsize=(8, 4))
        dc.plot_volcano_df(
            results_df,
            x='log2FoldChange',
            y='padj',
            top=20,
            figsize=(8, 4)
        )
        
        # Set title to indicate cell type
        plt.title(f"Volcano Plot for Cell Type: {cell_type}")
        
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()  # Close the plot to avoid display issues in the next iteration

print("PDF with all volcano plots saved as 'volcano_plots_all_cell_types.pdf'")

PDF with all volcano plots saved as 'volcano_plots_all_cell_types.pdf'


<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

...

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

## Pydeseq tumor vs normal FEMALE

In [132]:
pdata_tumor_normal_female

View of AnnData object with n_obs × n_vars = 696 × 17811
    obs: 'sample', 'standardized_cell_type', 'uicc_stage', 'ever_smoker', 'donor_id', 'origin', 'dataset', 'tumor_stage', 'study', 'platform', 'suspension_type', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'psbulk_n_cells', 'psbulk_counts'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    layers: 'psbulk_props'

In [133]:
pdata_tumor_normal_male

View of AnnData object with n_obs × n_vars = 518 × 17811
    obs: 'sample', 'standardized_cell_type', 'uicc_stage', 'ever_smoker', 'donor_id', 'origin', 'dataset', 'tumor_stage', 'study', 'platform', 'suspension_type', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'psbulk_n_cells', 'psbulk_counts'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    layers: 'psbulk_props'

In [70]:
pdata_tumor_normal_female[pdata_tumor_normal_female.obs[cell_type_class] == "regulatory T cell"].obs.origin

Guo_Zhang_2018_P1010_TTY_regulatory T cell         tumor-primary
Guo_Zhang_2018_P1118_TTH_regulatory T cell         tumor-primary
Guo_Zhang_2018_P1202_TTH_regulatory T cell         tumor-primary
Guo_Zhang_2018_P1202_TTR_regulatory T cell         tumor-primary
Guo_Zhang_2018_P1202_TTY_regulatory T cell         tumor-primary
                                                      ...       
UKIM-V-2_P5_tumor_primary_regulatory T cell        tumor-primary
UKIM-V-2_P7_normal_adjacent_regulatory T cell    normal-adjacent
UKIM-V-2_P7_tumor_primary_regulatory T cell        tumor-primary
UKIM-V_P1_normal_adjacent_regulatory T cell      normal-adjacent
UKIM-V_P1_tumor_primary_regulatory T cell          tumor-primary
Name: origin, Length: 74, dtype: object

In [71]:
import pdb
for contrast in contrasts:
    de_res = {}

    for ct in cell_types:
        print(ct)
        
        pb_ct = pdata_tumor_normal_female[pdata_tumor_normal_female.obs[cell_type_class] == ct].copy()

     
        #pdb.set_trace()
        if len(set(pb_ct.obs["origin"]).intersection([contrast["reference"], contrast["condition"]])) < 1:
             print(
                 "Not running DEseq for: "
                  + ct
                  + " : only present in: "
                  + str(set(pb_ct.obs["sex"]).intersection([contrast["reference"], contrast["condition"]]))
             )
             continue
        #pdb.set_trace()
                     
        dds = DeseqDataSet(
           adata=pb_ct,
           design_factors=[contrast["var"]],
           ref_level=[contrast["var"], 'normal-adjacent'],
           refit_cooks=True,
           n_cpus=cpus,
           )
             
        

        # Compute LFCs
        dds.deseq2()

 

        

        # Extract contrast
        stat_res = DeseqStats(
              dds,
              contrast=[contrast["var"],'tumor-primary', 'normal-adjacent'],
              #n_cpus=cpus,
          )
   
    

        # Compute Wald test
        stat_res.summary()

        # Shrink LFCs

        coeff = contrast["var"] + "_" + contrast["name"]
        stat_res.lfc_shrink(coeff=coeff)

        # Register cell type results
        de_res[ct] = stat_res.results_df
        de_res[ct]["standardized_cell_type"] = ct
        de_res[ct]["feature_name"] = stat_res.results_df.index.values

        de_res[ct].drop(columns=["feature_name"]).to_csv(
                Path(
                    contrast["res_dir"],
                    ct.replace(" ", "_") + "_" + contrast["name"] + "_deseq.tsv",
                ),
                sep="\t",
            )

        # Register results for current contrast
        contrast["de_res"] = de_res
        contrast["de_res_all"] = pd.concat([df.assign(cell_type=ct) for ct, df in de_res.items()])

B cell


Fitting size factors...
... done in 0.03 seconds.

Fitting dispersions...
... done in 1.89 seconds.

Fitting dispersion trend curve...
... done in 0.39 seconds.

Fitting MAP dispersions...
... done in 2.31 seconds.

Fitting LFCs...
... done in 1.81 seconds.

Calculating cook's distance...
... done in 0.08 seconds.

Replacing 85 outlier genes.

Fitting dispersions...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.03 seconds.

Fitting LFCs...
... done in 0.03 seconds.

Running Wald tests...
... done in 10.53 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
feature_name                                                                  
A1BG          6.836024       -0.096811  0.290096 -0.333722  0.738589  0.998775
A1BG-AS1      1.168325       -0.078309  0.423935 -0.184720  0.853449       NaN
A2M           1.288579       -0.575284  0.495898 -1.160086  0.246014       NaN
A2M-AS1       0.247093       -0.922256  0.751221 -1.227676  0.219569       NaN
A2ML1         0.052073       -1.773977  1.897850 -0.934730  0.349927       NaN
...                ...             ...       ...       ...       ...       ...
ZXDC          2.439321       -0.560718  0.462610 -1.212075  0.225484  0.928499
ZYG11A        0.217108       -0.436415  0.821729 -0.531093  0.595354       NaN
ZYG11B        3.527944       -0.286148  0.383959 -0.745257  0.456117  0.988955
ZYX           5.614568        0.189883  0.225259  0.8

/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
... done in 3.12 seconds.



Shrunk log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
              baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
feature_name                                                                  
A1BG          6.836024       -0.004149  0.095569 -0.333722  0.738589  0.998775
A1BG-AS1      1.168325        0.001322  0.120595 -0.184720  0.853449       NaN
A2M           1.288579       -0.008168  0.097222 -1.160086  0.246014       NaN
A2M-AS1       0.247093       -0.000984  0.116783 -1.227676  0.219569       NaN
A2ML1         0.052073       -0.001365  0.087317 -0.934730  0.349927       NaN
...                ...             ...       ...       ...       ...       ...
ZXDC          2.439321       -0.009504  0.093922 -1.212075  0.225484  0.928499
ZYG11A        0.217108        0.003583  0.122013 -0.531093  0.595354       NaN
ZYG11B        3.527944       -0.007244  0.095545 -0.745257  0.456117  0.988955
ZYX           5.614568        0.013493  0.1056

...

Shrunk log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
feature_name                                                            
A1BG          25.920354       -0.096844  0.183137 -1.357542  0.174609   
A1BG-AS1       1.295659        0.025356  0.239748  0.269529  0.787523   
A2M            9.845287       -0.004355  0.218801 -0.161217  0.871923   
A2M-AS1        0.645594       -0.010935  0.220984 -1.327870  0.184221   
A2ML1          0.015050        0.000010  0.248723 -0.550919  0.581689   
...                 ...             ...       ...       ...       ...   
ZXDC           1.277678       -0.013319  0.226521 -0.612614  0.540132   
ZYG11A         0.331280        0.017407  0.269614 -0.342529  0.731953   
ZYG11B         2.948444        0.055479  0.236746  1.155673  0.247815   
ZYX            4.748772       -0.031387  0.207016 -0.714831  0.474714   
ZZEF1          3.404421       -0.044202

Fitting size factors...
... done in 0.04 seconds.

Fitting dispersions...
... done in 2.48 seconds.

Fitting dispersion trend curve...
... done in 0.39 seconds.

Fitting MAP dispersions...
... done in 2.21 seconds.

Fitting LFCs...
... done in 2.05 seconds.

Calculating cook's distance...
... done in 0.08 seconds.

Replacing 1459 outlier genes.

Fitting dispersions...
... done in 0.32 seconds.

Fitting MAP dispersions...
... done in 0.23 seconds.

Fitting LFCs...
... done in 0.21 seconds.

Running Wald tests...
... done in 10.88 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
feature_name                                                            
A1BG           8.394548       -0.023311  0.335508 -0.069478  0.944609   
A1BG-AS1       1.646350        0.183794  0.459626  0.399877  0.689247   
A2M            1.120256       -0.538052  0.577106 -0.932329  0.351166   
A2M-AS1        0.964130       -0.977202  0.717689 -1.361596  0.173325   
A2ML1          0.039965       -2.118367  3.044130 -0.695886  0.486500   
...                 ...             ...       ...       ...       ...   
ZXDC           2.175864       -0.483790  0.414537 -1.167061  0.243186   
ZYG11A         0.057260       -2.055337  2.425106 -0.847525  0.396703   
ZYG11B         4.984575       -0.067816  0.385280 -0.176016  0.860281   
ZYX           19.014864       -0.347384  0.224488 -1.547451  0.121754   
ZZEF1          6.800469       -0.189774  0.282

/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in multiply
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/home/kvalem/.conda/envs/crc_atlas/lib/python3.11/site-packages/pydeseq2/utils.py:1260: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
... done in 6.10 seconds.



Shrunk log2 fold change & Wald test p-value: origin tumor-primary vs normal-adjacent
               baseMean  log2FoldChange     lfcSE      stat    pvalue  \
feature_name                                                            
A1BG           8.394548   -1.880110e-07  0.001673 -0.069478  0.944609   
A1BG-AS1       1.646350    3.721156e-06  0.002116  0.399877  0.689247   
A2M            1.120256   -5.311020e-07  0.001730 -0.932329  0.351166   
A2M-AS1        0.964130   -1.028639e-06  0.001682 -1.361596  0.173325   
A2ML1          0.039965    1.276819e-07  0.002271 -0.695886  0.486500   
...                 ...             ...       ...       ...       ...   
ZXDC           2.175864   -2.659679e-06  0.001786 -1.167061  0.243186   
ZYG11A         0.057260   -6.464404e-08  0.001548 -0.847525  0.396703   
ZYG11B         4.984575   -3.949415e-07  0.001750 -0.176016  0.860281   
ZYX           19.014864   -7.261606e-06  0.001601 -1.547451  0.121754   
ZZEF1          6.800469   -2.570082e-06

In [131]:
contrast["de_res_all"].to_csv("dea_female_tumor_vs_normal.csv")

In [73]:
resDir = resDir_tables

In [74]:
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Loop through each cell type and save its data to CSV
for cell_type in cell_types:
    # Filter the DataFrame for the current cell type
    ct_all_deg = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]
    
    # Remove spaces from cell_type for the filename
    cell_type_filename = cell_type.replace(" ", "")
    
    # Save the full data for the cell type
    filename_deg = f"{cell_type_filename}_deg_80cells.csv"
    filepath_deg = os.path.join(resDir, filename_deg)
    ct_all_deg.to_csv(filepath_deg, index=False)
    print(f"Saved full {cell_type} data to {filepath_deg}")
    
    # Apply additional filtering for significant results
    filtered_df = ct_all_deg[(ct_all_deg['padj'] < 0.1) & (ct_all_deg['log2FoldChange'].abs() > 1)]
    
    # Save the filtered significant data for the cell type
    filename_sig_deg = f"{cell_type_filename}_sig_deg.csv"
    filepath_sig_deg = os.path.join(resDir, filename_sig_deg)
    filtered_df.to_csv(filepath_sig_deg, index=False)
    print(f"Saved significant {cell_type} data to {filepath_sig_deg}")

Saved full B cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/Bcell_deg.csv
Saved significant B cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/Bcell_sig_deg.csv
Saved full CD4+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD4+Tcell_deg.csv
Saved significant CD4+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD4+Tcell_sig_deg.csv
Saved full CD8+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD8+Tcell_deg.csv
Saved significant CD8+ T cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/CD8+Tcell_sig_deg.csv
Saved full dendritic cell data to /data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/dendriticcell_deg.csv
Saved significant d

In [75]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming `contrast` and `dc` are already defined and set up in your environment
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Create a PDF file to save all volcano plots
with PdfPages(f"{resDir_figures}/volcano_plots_all_cell_types.pdf") as pdf:
    for cell_type in cell_types:
        # Filter results for the current cell type
        results_df = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]

        # Generate the volcano plot
        plt.figure(figsize=(8, 4))
        dc.plot_volcano_df(
            results_df,
            x='log2FoldChange',
            y='padj',
            top=20,
            figsize=(8, 4)
        )
        
        # Set title to indicate cell type
        plt.title(f"Volcano Plot for Cell Type: {cell_type}")
        
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()  # Close the plot to avoid display issues in the next iteration

print("PDF with all volcano plots saved as 'volcano_plots_all_cell_types.pdf'")

PDF with all volcano plots saved as 'volcano_plots_all_cell_types.pdf'


<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

...

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

In [135]:
pdata_tumor_normal_female.obs

,sample,standardized_cell_type,uicc_stage,ever_smoker,donor_id,origin,dataset,tumor_stage,study,platform,...,tissue_ontology_term_id,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,psbulk_n_cells,psbulk_counts
He_Fan_2021_LUAD1_B cell,He_Fan_2021_LUAD1,B cell,I,no,He_Fan_2021_P1,tumor-primary,He_Fan_2021,early,He_Fan_2021,10x,...,UBERON:0002048,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,48-year-old human stage,89.0,118337.0
He_Fan_2021_LUAD3_B cell,He_Fan_2021_LUAD3,B cell,I,yes,He_Fan_2021_P3,tumor-primary,He_Fan_2021,early,He_Fan_2021,10x,...,UBERON:0002048,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,58-year-old human stage,45.0,44666.0
He_Fan_2021_LUAD5_B cell,He_Fan_2021_LUAD5,B cell,II,no,He_Fan_2021_P5,tumor-primary,He_Fan_2021,early,He_Fan_2021,10x,...,UBERON:0002048,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,56-year-old human stage,328.0,769382.0
He_Fan_2021_N1_B cell,He_Fan_2021_N1,B cell,I,no,He_Fan_2021_P1,normal-adjacent,He_Fan_2021,early,He_Fan_2021,10x,...,UBERON:0002048,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,48-year-old human stage,75.0,102672.0
He_Fan_2021_N3_B cell,He_Fan_2021_N3,B cell,I,yes,He_Fan_2021_P3,normal-adjacent,He_Fan_2021,early,He_Fan_2021,10x,...,UBERON:0002048,10x 3' v2,lung adenocarcinoma,Homo sapiens,female,lung,unknown,58-year-old human stage,21.0,26543.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UKIM-V-2_P5_tumor_primary_regulatory T cell,UKIM-V-2_P5_tumor_primary,regulatory T cell,II,yes,UKIM-V-2_P5,tumor-primary,UKIM-V-2,early,UKIM-V,BD-Rhapsody,...,UBERON:0002048,BD Rhapsody Whole Transcriptome Analysis,lung adenocarcinoma,Homo sapiens,female,lung,unknown,62-year-old human stage,183.0,466299.0
UKIM-V-2_P7_normal_adjacent_regulatory T cell,UKIM-V-2_P7_normal_adjacent,regulatory T cell,III,yes,UKIM-V-2_P7,normal-adjacent,UKIM-V-2,advanced,UKIM-V,BD-Rhapsody,...,UBERON:0002048,BD Rhapsody Whole Transcriptome Analysis,lung adenocarcinoma,Homo sapiens,female,lung,unknown,69-year-old human stage,12.0,26681.0
UKIM-V-2_P7_tumor_primary_regulatory T cell,UKIM-V-2_P7_tumor_primary,regulatory T cell,III,yes,UKIM-V-2_P7,tumor-primary,UKIM-V-2,advanced,UKIM-V,BD-Rhapsody,...,UBERON:0002048,BD Rhapsody Whole Transcriptome Analysis,lung adenocarcinoma,Homo sapiens,female,lung,unknown,69-year-old human stage,90.0,188984.0
UKIM-V_P1_normal_adjacent_regulatory T cell,UKIM-V_P1_normal_adjacent,regulatory T cell,I,yes,UKIM-V_P1,normal-adjacent,UKIM-V,early,UKIM-V,BD-Rhapsody,...,UBERON:0002048,BD Rhapsody Whole Transcriptome Analysis,lung adenocarcinoma,Homo sapiens,female,lung,unknown,65-year-old human stage,34.0,548293.0


In [139]:
import pandas as pd
import scanpy as sc

def extract_b_cell_data(adata, gender):
    """
    Extracts B cell data from an AnnData object and returns count matrix and metadata.
    
    Parameters:
        adata (AnnData): The AnnData object
        gender (str): Gender label for samples ("female" or "male")

    Returns:
        count_matrix (pd.DataFrame): Gene expression matrix
        samplesheet (pd.DataFrame): Metadata for each sample
    """
    # Subset to only B cells
    b_cell_adata = adata[adata.obs["standardized_cell_type"] == "B cell"].copy()

    # Extract count matrix (assuming raw counts are stored in .X or .layers["counts"])
    count_matrix = pd.DataFrame(
        b_cell_adata.X.A if isinstance(b_cell_adata.X, np.matrix) else b_cell_adata.X,
        index=b_cell_adata.obs_names,
        columns=b_cell_adata.var_names
    )
    
    # Extract metadata for the samplesheet
    samplesheet = b_cell_adata.obs.copy()
    samplesheet["gender"] = gender  # Add gender column
    
    return count_matrix, samplesheet

# Process female and male datasets
count_matrix_female, samplesheet_female = extract_b_cell_data(pdata_tumor_normal_female, "female")
count_matrix_male, samplesheet_male = extract_b_cell_data(pdata_tumor_normal_male, "male")

# Combine results from both genders
final_count_matrix = pd.concat([count_matrix_female, count_matrix_male], axis=0)
final_samplesheet = pd.concat([samplesheet_female, samplesheet_male], axis=0)

# Save to CSV
final_count_matrix.T.to_csv("B_cell_count_matrix.csv")
final_samplesheet.to_csv("B_cell_samplesheet.csv")

print("Count matrix and samplesheet saved successfully.")


Count matrix and samplesheet saved successfully.
